In [1]:
# sacado de la documentación, yo no lo he ejecutado.
# %pip install llama-index-llms-openai llama-index-embeddings-openai

# estos son los módulos que tengo instalados
# pip install llama-index
# pip install llama-index-embeddings-huggingface
# pip install llama-index-llms-ollama
# pip install matplotlib


In [2]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [3]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "TU KEY"
# openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Response
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    CorrectnessEvaluator,
    GuidelineEvaluator,
    SemanticSimilarityEvaluator
)
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd

pd.set_option("display.max_colwidth", 0)

In [5]:
# gpt-4
# gpt4 = OpenAI(temperature=0, model="gpt-4")
gpt4 = OpenAI(temperature=0, model="gpt-3.5-turbo")

faithfulness_gpt4 = FaithfulnessEvaluator(llm=gpt4)
relevancy_gpt4 = RelevancyEvaluator(llm=gpt4)
correctness_gpt4 = CorrectnessEvaluator(llm=gpt4)
semantic = SemanticSimilarityEvaluator()  # coge el modelo de embedings de settings si no se le pasa ninguno.

In [134]:
from llama_index.core.llama_dataset import download_llama_dataset, LabelledRagDataset

# rag_dataset = LabelledRagDataset.from_json("./eval_data/rag_data/paul_graham/rag_dataset.json")  # 44 queries.
# documents = SimpleDirectoryReader(input_dir="./eval_data/rag_data/paul_graham/source_files").load_data()

# rag_dataset = LabelledRagDataset.from_json("./eval_data/rag_data/Blockchain/rag_dataset.json")  # 58 queries.
# documents = SimpleDirectoryReader(input_dir="./eval_data/rag_data/Blockchain/source_files").load_data()

# rag_dataset = LabelledRagDataset.from_json("./eval_data/rag_data/Alexnet/rag_dataset.json")  # 160 queries.
# documents = SimpleDirectoryReader(input_dir="./eval_data/rag_data/Alexnet/source_files").load_data()

rag_dataset = LabelledRagDataset.from_json("./eval_data/rag_data/Covid/rag_dataset.json")  # 316 queries.
documents = SimpleDirectoryReader(input_dir="./eval_data/rag_data/Covid/source_files").load_data()

# rag_dataset = LabelledRagDataset.from_json("./eval_data/rag_data/Llama2Paper/rag_dataset.json")  # 100 queries.
# documents = SimpleDirectoryReader(input_dir="./eval_data/rag_data/Llama2Paper/source_files").load_data()


# SE QUEDA PILLADO porque son muchos documentos creo

# rag_dataset = LabelledRagDataset.from_json("./eval_data/rag_data/FinanceBench/rag_dataset.json")  # 98 queries.
# documents = SimpleDirectoryReader(input_dir="./eval_data/rag_data/FinanceBench/source_files").load_data()

In [135]:
# en el caso de que quisieramos generar un dataset de queries a raíz de los documentos

# from llama_index.core.evaluation import DatasetGenerator

# dataset_generator = DatasetGenerator.from_documents(documents, llm=llm)

# qas = dataset_generator.generate_dataset_from_nodes(num=3)

In [136]:

# rag_dataset = LabelledRagDataset.from_json("./eval_data/rag_data/FinanceBench/rag_dataset.json")  # 98 queries.
# documents = SimpleDirectoryReader(input_dir="./eval_data/rag_data/FinanceBench/source_files").load_data()rag_dataset_pandas = rag_dataset.to_pandas()
# queries = rag_dataset_pandas["query"]
# reference_answers = rag_dataset_pandas["reference_answer"]
# queries = queries[:20]
# reference_answers = reference_answers[:20].to_list()# create vector index
# splitter = SentenceSplitter(chunk_size=512)
# vector_index = VectorStoreIndex.from_documents(
#     documents, transformations=[splitter]
# )

In [137]:
rag_dataset_pandas = rag_dataset.to_pandas()
queries = rag_dataset_pandas["query"]
reference_answers = rag_dataset_pandas["reference_answer"]

In [138]:
# este paso lo hago simplemente para reducir el tamaño del dataset,
# para que no tarde mucho la evaluación cada vez que hago pruebas durante el desarrollo

queries = queries[:20]
reference_answers = reference_answers[:20].to_list()

In [139]:
print(len(queries))
print(len(reference_answers))

20
20


In [140]:
# # aquí crear el modelo que quieras evaluar

# llm = OpenAI(temperature=0.3, model="gpt-3.5-turbo")

In [141]:
# !pip install sentencepiece
# !pip install protobuf
# !conda install conda-forge::triton

In [142]:
# aquí crear el modelo que quieras evaluar

from llama_index.llms.ollama import Ollama

llm = Ollama(model="internlm2", request_timeout=360.0)

In [143]:
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# embed_model.__class__

In [144]:
# create vector index
splitter = SentenceSplitter(chunk_size=512)
vector_index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter]
)

In [145]:
from llama_index.core.evaluation import BatchEvalRunner

# ponemos todos los evaluadores que queremos usar aquí.
runner = BatchEvalRunner(
    {"correctness": correctness_gpt4,
     "faithfulness": faithfulness_gpt4,
     "relevancy": relevancy_gpt4,
     "semantic": semantic},
    workers=8,
)

# he corregido una cosa de aquí del ejemplo, que no estaba (llm=llm), supongo que por error.
eval_results = await runner.aevaluate_queries(
    vector_index.as_query_engine(llm=llm),
    queries=queries,
    reference=reference_answers,
)

In [146]:
# print(eval_results)
print(len(eval_results["correctness"]))

20


In [147]:
# total_results = eval_results.copy()

In [71]:
def combinar_diccionarios(dict1, dict2):
    resultado = {}
    for key in dict1:
        if key in dict2:
            # Combina los arrays de ambas claves en un solo array
            resultado[key] = dict1[key] + dict2[key]
    return resultado

In [148]:
total_results = combinar_diccionarios(total_results, eval_results)
print(len(total_results["correctness"]))

100


In [122]:
# print(total_results)

In [149]:
print(eval_results.keys())

print(eval_results["correctness"][0].dict().keys())

print(eval_results["correctness"][0].passing)
print(eval_results["correctness"][0].response)
print(eval_results["correctness"][0].contexts)

dict_keys(['correctness', 'faithfulness', 'relevancy', 'semantic'])
dict_keys(['query', 'contexts', 'response', 'passing', 'feedback', 'score', 'pairwise_source', 'invalid_result', 'invalid_reason'])
True
The article details several potential early signs of Covid-19 infection. These include sore throat, fever, chills, muscle aches, gastrointestinal disturbances such as diarrhea and nausea, and changes in sense of smell or taste. Some individuals may experience painful red and purple lesions on their fingers and toes, which are referred to as "Covid toe." Symptoms can vary widely among different people; some might not show many symptoms at all.

More serious cases have been associated with inflammation and organ damage even without difficulty breathing. There has also been a noted incidence of dangerous blood clots, strokes, and brain impairments in severe Covid-19 infections.

None


In [75]:
# he modificado la del ejemplo para que use result.score en vez de result.passing. en algunos tipos de evaluación sale un resultado más preciso
def get_eval_results(key, eval_results):
    results = eval_results[key]
    sum_score = 0
    count = 0
    for result in results:
        if result.score is not None:  # Verifica si score no es None
            sum_score += result.score
            count += 1
    if count == 0:
        print(f"{key} Score: No valid scores")
        return None
    score = sum_score / count
    print(f"{key} Score: {score}")
    return score

In [150]:
score = get_eval_results("faithfulness", eval_results)

faithfulness Score: 0.95


In [151]:
score = get_eval_results("relevancy", eval_results)

relevancy Score: 0.85


In [152]:
score = get_eval_results("correctness", eval_results)

correctness Score: 3.90625


In [153]:
score = get_eval_results("semantic", eval_results)

semantic Score: 0.8915432709518762


In [162]:
print(f"model: {llm.model}")
score_faithfulness = get_eval_results("faithfulness", eval_results)
score_relevancy = get_eval_results("relevancy", eval_results)
score_correctness = get_eval_results("correctness", eval_results)
score_semantic = get_eval_results("semantic", eval_results)

# print(f"score faithfulness: {score_faithfulness}")

model: internlm2
faithfulness Score: 0.95
relevancy Score: 0.85
correctness Score: 3.90625
semantic Score: 0.8915432709518762


In [129]:
print(eval_results["faithfulness"][:1])

[EvaluationResult(query='What are some traditional application domains where deep learning has been successfully applied?', contexts=['This is the strong point of deep learning against \ntraditional machine learning approaches. The following table \nshows the different feature -based  learn ing approaches with \ndifferent learning steps.   \n       \nFig. 3.  Where to apply DL approaches  \nC. When and where to apply DL  \nDL is employed in several situations where machine \nintelligence would be useful (see Fig. 3): \n1. Absence of a human expert (navigation on  Mars)  \n2. Human s are unable to explain their expertise (speech \nrecognition, vision and language understanding)  \n3. The solution to the problem changes over time (tracking, \nweather prediction, preference, stock, price prediction)  \n4. Solutions need to be adapted to the particular case s \n(biometrics, personalization) . \n5. The problem size is too vast for our limited reasoning \ncapabilities (calculation webpage ra

In [130]:
print(eval_results["relevancy"][:1])

[EvaluationResult(query='What are some traditional application domains where deep learning has been successfully applied?', contexts=['This is the strong point of deep learning against \ntraditional machine learning approaches. The following table \nshows the different feature -based  learn ing approaches with \ndifferent learning steps.   \n       \nFig. 3.  Where to apply DL approaches  \nC. When and where to apply DL  \nDL is employed in several situations where machine \nintelligence would be useful (see Fig. 3): \n1. Absence of a human expert (navigation on  Mars)  \n2. Human s are unable to explain their expertise (speech \nrecognition, vision and language understanding)  \n3. The solution to the problem changes over time (tracking, \nweather prediction, preference, stock, price prediction)  \n4. Solutions need to be adapted to the particular case s \n(biometrics, personalization) . \n5. The problem size is too vast for our limited reasoning \ncapabilities (calculation webpage ra

In [131]:
print(eval_results["correctness"][:1])

[EvaluationResult(query='What are some traditional application domains where deep learning has been successfully applied?', contexts=None, response="Deep learning applications have transcended conventional boundaries in various fields. Here's a concise list of some significant areas:\n\n1. **Computer Vision**:\n   - Object localization and detection.\n   - Image or video captioning.\n\n2. **Multimedia Entertainment**: \n   - Media processing, including generating content like images from text (text-to-image synthesis).\n\n3. **Autonomous Vehicles**: \n   - Vehicle segmentation in autonomous cars to enhance safety features such as collision avoidance systems.\n\n4. **Language Processing and Natural Language Understanding**:\n   - Machine translation.\n   - Speech recognition tasks involving voice-based commands or interaction with AI assistants like Siri, Alexa etc.\n\n5. **Healthcare and Medicine**: \n   - Brian cancer detection using medical imaging data analysis techniques powered by

In [132]:
print(eval_results["semantic"][:1])

[EvaluationResult(query=None, contexts=None, response=None, passing=True, feedback='Similarity score: 0.9386481964361731', score=0.9386481964361731, pairwise_source=None, invalid_result=False, invalid_reason=None)]
